In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator






In [8]:
# Load the trained Inception V3 model
model = tf.keras.models.load_model("watermelon_model.keras")  # Replace with your model path

In [27]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [36]:


# Function to preprocess the image
def preprocess_image(image_path):
    img = Image.open(image_path).resize((256, 256))  # Resize to 299x299 for InceptionV3
    img_array = image.img_to_array(img)  # Convert image to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = preprocess_input(img_array)  # Preprocess image for InceptionV3
    return img_array

# # Define the function to predict the disease
# def predict_disease(image_path, threshold=0.8, top_n_classes=3):
#     # Preprocess the input image
#     input_image = preprocess_image(image_path)
    
#     # Get predictions from the model
#     predictions = model.predict(input_image)
    
#     # Calculate softmax probabilities
#     probabilities = tf.nn.softmax(predictions[0])  # Apply softmax to get probabilities
#     max_confidence = np.max(probabilities)  # Get the max confidence
#     predicted_class = np.argmax(probabilities)  # Get the predicted class index
    
#     # Print the confidence scores for debugging
#     #print(f"Predicted Class Index: {predicted_class}, Max Confidence: {max_confidence}")
    
#     # Get the top N classes with their probabilities
#     top_n_classes_indices = tf.argsort(probabilities, direction='DESCENDING')[:top_n_classes]
#     top_n_probs = tf.gather(probabilities, top_n_classes_indices)
    
#     # Print the top N class probabilities for debugging
#     #print(f"Top {top_n_classes} Class Indices: {top_n_classes_indices.numpy()}")
#     #print(f"Top {top_n_classes} Class Probabilities: {top_n_probs.numpy()}")
    
#     # Check for OOD (confidence below the threshold)
#     if any(prob > threshold for prob in top_n_probs.numpy()):
#         # Use the dynamic class indices from the validation set
#         validation_dir = r"C:\Users\Public\Documents\watermelon" # Update with the path to your validation dataset
        
#         # Load validation data
#         datagen = ImageDataGenerator(rescale=1./255)  # Normalize pixel values
#         validation_generator = datagen.flow_from_directory(
#             validation_dir,
    #         target_size=(256, 256),  # Adjust to your model's input size
    #         batch_size=32,
    #         class_mode='categorical',
    #         shuffle=False  # Important to maintain order for confusion matrix
    #     )
        
    #     # Get class labels (mapping indices to class names)
    #     class_indices = validation_generator.class_indices
    #     class_names = {v: k for k, v in class_indices.items()}  # Reverse the mapping
    #     predicted_class_label = class_names[predicted_class]

    #     print(f"Predicted Class Index: {predicted_class}")
    #     #print(f"Confidence: {max_confidence:.2f}")
      
        
    #     # Return the predicted class label and confidence
    #     return f"Predicted Class Label: {predicted_class_label}"
    # else:
    #     return "Irrelevant Image"
def predict_disease(image_path, threshold=0.8, top_n_classes=3):
    # Use the dynamic class indices from the validation set
    validation_dir = r"C:\Users\Public\Documents\watermelon"  # Update with the path to your validation dataset

    # Load validation data
    datagen = ImageDataGenerator(rescale=1./255)  # Normalize pixel values
    validation_generator = datagen.flow_from_directory(
        validation_dir,
        target_size=(256, 256),  # Adjust to your model's input size
        batch_size=32,
        class_mode='categorical',
        shuffle=False  # Important to maintain order for confusion matrix
    )

    # Get class labels (mapping indices to class names)
    class_indices = validation_generator.class_indices
    print("Class Labels:")
    for index, label in class_indices.items():
        print(f"{index}: {label}")
    
    # Preprocess the input image
    input_image = preprocess_image(image_path)

    # Get predictions from the model
    predictions = model.predict(input_image)

    # Calculate softmax probabilities
    probabilities = tf.nn.softmax(predictions[0])  # Apply softmax to get probabilities
    max_confidence = np.max(probabilities)  # Get the max confidence
    predicted_class = np.argmax(probabilities)  # Get the predicted class index
    
    # Get the top N classes with their probabilities
    top_n_classes_indices = tf.argsort(probabilities, direction='DESCENDING')[:top_n_classes]
    top_n_probs = tf.gather(probabilities, top_n_classes_indices)
    
    # Check for OOD (confidence below the threshold)
    if any(prob > threshold for prob in top_n_probs.numpy()):
        # Directly fetch the predicted class label using the class index
        predicted_class_label = list(class_indices.keys())[list(class_indices.values()).index(predicted_class)]

        # Print the prediction
        print(f"Predicted Class Index: {predicted_class}")
        #print(f"Confidence: {max_confidence:.2f}")
        
        # Return the predicted class label and confidence
        return f"Predicted Class Label: {predicted_class_label}"
    else:
        return "Irrelevant Image"

In [38]:
# Example of using the function
image_path = r"C:\Users\Public\Documents\watermelon\Anthracnose\IMG_4217_shifted.jpg"  # Replace with the new image path
result = predict_disease(image_path, threshold=0.3)
print(result)

Found 120 images belonging to 4 classes.
Class Labels:
Anthracnose: 0
Downy_Mildew: 1
Healthy: 2
Mosaic_Virus: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
Predicted Class Index: 0
Predicted Class Label: Anthracnose
